In [1]:
import SimpleITK as sitk
import os
import shutil
import numpy as np

In [2]:
# input_directory
model_directory = "C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST/"
template_file = os.path.join( model_directory,  "25um_V3", "AvgfMOSTAtlas25umV3.nii.gz")

In [3]:
# brain inside/outside mask from Maitham's analysis
mask_file = os.path.join( model_directory, "outer_boundaries/surface_voxels_v02/output", "fmost_inner_outer_mask.nii.gz")

In [4]:
# output directory
output_directory = 'landmarks_v05/fmost_annotation'
input_file = os.path.join( output_directory, 'fmost_landmarks_one_sided.nrrd' )
flipped_file = os.path.join( output_directory, 'fmost_template_flipped.nii.gz' )
output_file = os.path.join( output_directory, 'fmost_landmarks_double_sided.nrrd')
filled_file = os.path.join( output_directory, 'fmost_landmarks_double_sided_filled.nii.gz')

In [5]:
# flip the template image for validation
template = sitk.ReadImage( template_file )
resampled = sitk.Flip( template, (True,False,False), False )
resampled.CopyInformation( template )

sitk.WriteImage( resampled, flipped_file, True )

In [6]:
# flip and combine the annotation file
annotation = sitk.ReadImage( input_file )
resampled = sitk.Flip( annotation, (True,False,False), False )
resampled.CopyInformation( template )
annotation.CopyInformation( template )

resampled = sitk.Maximum( annotation, resampled )
sitk.WriteImage( resampled, output_file, True )

In [7]:
# combine manual and automated annotation into a single file
mask = sitk.ReadImage( mask_file )

xarr = sitk.GetArrayFromImage( mask )
yarr = sitk.GetArrayFromImage( resampled )
idx = np.where( yarr > 0 )
xarr[idx] = yarr[idx]

annotation = sitk.GetImageFromArray( xarr )
annotation.CopyInformation( template )

sitk.WriteImage( annotation, filled_file, True)


In [8]:
# copy the label description files to output directory
label_file = os.path.join( output_directory, '../ccf_annotation/landmark_description.txt')
output_file = os.path.join( output_directory, 'landmark_description.txt')

shutil.copyfile(label_file, output_file)

'landmarks_v05/fmost_annotation\\landmark_description.txt'